<a href="https://colab.research.google.com/github/fahernkhan/Machine-Learning-Deployment/blob/main/Export_Data_ke_Training_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
 
import tensorflow as tf
 
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

Selanjutnya gunakan Pandas DataFrame untuk membaca file CSV

In [2]:
url= 'https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv'
df = pd.read_csv(url)

Setelah data telah dalam format dataframe, Split dataset menjadi training, test, dan validation. Selain itu, kita juga perlu mendefinisikan kolom target.

In [3]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


In [4]:
#Membuat kolom target
# 0 = low; 1 = high
df['target'] = np.where(df['price'] <= 1000, 0, 1)
# Drop un-used columns.
df = df.drop(columns=['price'])

Kemudian bungkus dataframe dengan tf.data agar memungkinkan untuk menggunakan feature column sebagai jembatan dari dataframe menjadi fitur yang digunakan untuk train model. Jika file CSV nya sangat besar, kita harus menggunakan tf.data untuk membaca file tersebut dari harddisk.

In [5]:
# Cara untuk membuat dataset tf.data dari pandas dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = df.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds
batch_size = 10 #bath ukuran kecil untuk demonstrasi
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

Kemudian buat feature layer sebagai input ke dalam model tf.Keras.

In [6]:
#Pilih feature column mana yang akan digunakan
feature_columns = []
# numeric column
for header in ['carat', 'depth', 'x', 'y', 'z']:
  feature_columns.append(feature_column.numeric_column(header))
 
#membuat feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


Terakhir tulislah kode untuk create, compile, dan train model.

In [7]:
#create, compile, and train the model
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])
 
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
 
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10


1675/1686 [============================>.] - ETA: 0s - loss: 0.2156 - accuracy: 0.9065

1686/1686 [==============================] - 14s 7ms/step - loss: 0.2157 - accuracy: 0.9066 - val_loss: 0.1587 - val_accuracy: 0.9374
Epoch 2/10
1686/1686 [==============================] - 11s 6ms/step - loss: 0.1568 - accuracy: 0.9347 - val_loss: 0.1455 - val_accuracy: 0.9377
Epoch 3/10
1686/1686 [==============================] - 10s 6ms/step - loss: 0.1529 - accuracy: 0.9354 - val_loss: 0.1539 - val_accuracy: 0.9381
Epoch 4/10
1686/1686 [==============================] - 12s 7ms/step - loss: 0.1494 - accuracy: 0.9363 - val_loss: 0.1417 - val_accuracy: 0.9378
Epoch 5/10
1686/1686 [==============================] - 12s 7ms/step - loss: 0.1486 - accuracy: 0.9363 - val_loss: 0.1397 - val_accuracy: 0.9379
Epoch 6/10
1686/1686 [==============================] - 11s 6ms/step - loss: 0.1465 - accuracy: 0.9365 - val_loss: 0.1405 - val_accuracy: 0.9380
Epoch 7/10
1686/1686 [==============================] - 8s 5ms/step - loss: 0.1442 - accuracy: 0.9367 - val_loss: 0.1421 - val_accuracy: 0.93